# ibm_sql_query

Execute arbitrary SQL queries against CSV and PARQUET files using IBM Cloud SQL Query and Cloud Object Store

In [ ]:
!pip install ibmcloudsql==0.4.29

In [ ]:
import glob
import logging
import ibmcloudsql
from ibmcloudsql import SQLQuery
import os
import shutil
import sys
import re

In [ ]:
# IBM Cloud API key (alternative to token)
api_key = os.environ.get('api_key')

# IBM Cloud Token (alternative to API key)
token = os.environ.get('token')

# URI of resulting file (example: cos://s3.eu-de.cloud-object-storage.appdomain.cloud/cos-rkie-sqlquery-test/result)
output_result_file_uri = os.environ.get('output_result_file_uri')

# (unique) Custom Resource Name (CRN) of IBM SQL Query Service
sql_query_crn = os.environ.get('sql_query_crn')

# sql statement to execute ()
sql = os.environ.get('sql')

In [ ]:
token = 'eyJraWQiOiIyMDIxMTExNzA4MjAiLCJhbGciOiJSUzI1NiJ9.eyJpYW1faWQiOiJJQk1pZC0yNzAwMDI1NzNZIiwiaWQiOiJJQk1pZC0yNzAwMDI1NzNZIiwicmVhbG1pZCI6IklCTWlkIiwic2Vzc2lvbl9pZCI6IkMtMjA1YmU4NTAtMTIwNS00NjRjLTlhMjMtNzVhZjBkNzJiMDZiIiwianRpIjoiMTEwMzhiMDEtMTdiMC00NjgwLTk3NDEtYmFjYjAxNDkyMzc3IiwiaWRlbnRpZmllciI6IjI3MDAwMjU3M1kiLCJnaXZlbl9uYW1lIjoiUm9tZW8iLCJmYW1pbHlfbmFtZSI6IktpZW56bGVyIiwibmFtZSI6IlJvbWVvIEtpZW56bGVyIiwiZW1haWwiOiJyb21lby5raWVuemxlckBjaC5pYm0uY29tIiwic3ViIjoicm9tZW8ua2llbnpsZXJAY2guaWJtLmNvbSIsImF1dGhuIjp7InN1YiI6InJvbWVvLmtpZW56bGVyQGNoLmlibS5jb20iLCJpYW1faWQiOiJJQk1pZC0yNzAwMDI1NzNZIiwibmFtZSI6IlJvbWVvIEtpZW56bGVyIiwiZ2l2ZW5fbmFtZSI6IlJvbWVvIiwiZmFtaWx5X25hbWUiOiJLaWVuemxlciIsImVtYWlsIjoicm9tZW8ua2llbnpsZXJAY2guaWJtLmNvbSJ9LCJhY2NvdW50Ijp7ImJvdW5kYXJ5IjoiZ2xvYmFsIiwidmFsaWQiOnRydWUsImJzcyI6IjliMTNiODU3YTMyMzQxYjcxNjcyNTVkZTcxNzE3MmY1IiwiaW1zX3VzZXJfaWQiOiI4MDgwMjIyIiwiaW1zIjoiMjAzMjQ3NCJ9LCJpYXQiOjE2Mzc4NDc4NTUsImV4cCI6MTYzNzg0OTA1NSwiaXNzIjoiaHR0cHM6Ly9pYW0uY2xvdWQuaWJtLmNvbS9pZGVudGl0eSIsImdyYW50X3R5cGUiOiJ1cm46aWJtOnBhcmFtczpvYXV0aDpncmFudC10eXBlOnBhc3Njb2RlIiwic2NvcGUiOiJpYm0gb3BlbmlkIiwiY2xpZW50X2lkIjoiYngiLCJhY3IiOjEsImFtciI6WyJwd2QiXX0.VpQginD2Kxukgh7wdDdkr3fHbVGJ0-NhyaxW1w3JjT1rC1jB7MG_ZZ59xCvL0nhMVB0V5skRzr-1WBR9qxOGh6ExgHu-jRBd5JnQhfUT2ZKGyP5R2HzSUj6-jSUnwfFy93MetdljGDxybStSvQAUw8NiAtMv_E12JljZtwj6QtFHcY20mpMOzL1CoV6ef1nCOkAla0NrKdAfI873CVNVG1QPlAVhHmuzJ6YogmHlfKXlu_RHfJ8JNZSWeJQuXYmBIi5BcqoxPoLLqfngE-A4dn5d9g1C50h15PLX8oCyWEfXSIz9Iccvb709Hgr7a0eiIA9ADm5JbgnL8znbqNYS1A'
output_result_file_uri = 'cos://s3.eu-de.cloud-object-storage.appdomain.cloud/cos-rkie-sqlquery-test/result'
sql_query_crn = 'crn:v1:bluemix:public:sql-query:us-south:a/9b13b857a32341b7167255de717172f5:f9dd6c9e-b24b-4506-819e-e038c92339e4::'
sql='SELECT * FROM cos://eu-de/cos-rkie-sqlquery-test/12613-0004_flat-2.csv STORED AS CSV'

In [ ]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [ ]:
if token is None:
    sqlClient = SQLQuery(api_key, sql_query_crn)
else:
    sqlClient = SQLQuery(api_key=None, token=token, instance_crn=sql_query_crn)

sqlClient.run_sql(sql + ' INTO {} STORED AS CSV'.format(output_result_file_uri))